In [63]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
pd.set_option("display.max_columns", 180)
from tqdm.notebook import tqdm
from pathlib import Path
from IPython.display import display

In [49]:
def normalize(data):
    df=data.copy()
    for column in df.columns[11:]:
        minim = df[column].min()
        maxim = df[column].max()
        df[column] = (df[column] - minim) / (maxim - minim)
    return df

In [50]:
def compute_similarity(data, player, min_minutes=100):
    df = data[data.minutesPlayed>=min_minutes].reset_index(drop=True)
    df = normalize(df)
    j = df[df['playerId']==player].index.values[0]    
    similarity = df.copy()
    for row in range(0,len(df)):
        for column in df.columns[11:]:
            similarity.at[row,column] = abs(df.loc[row,column] - df.loc[j,column])
            
    similarity['Distance']= similarity.iloc[:,11:].mean(axis=1) 
    similarity['SimilarityIndex'] = round((1 - similarity['Distance'])*100, 3)
    similarity = similarity[['playerId', 
                             'shortName', 
                             'team', 
                             'position', 
                             'age',
                             'height', 
                             'foot', 
                             'minutesPlayed',
                             'marketValue', 
                             'goals', 
                             'minutesPlayed', 
                             'SimilarityIndex']]
    similarity = similarity.sort_values(by='SimilarityIndex', ascending=False).iloc[1:,:].reset_index(drop=True)
    return similarity

In [51]:
PATH = Path('C:/Users/gmanf/OneDrive/Documents/GitHub/Published-Studies/Playing Style Soccer/data/')

In [52]:
df = pd.read_csv(PATH / 'full_data.csv', delimiter=',')

## Simple program that allows to search Top N most similar players to a reference player

In [72]:
i=0
while i<1:
    j=0
    while j<1:
        player_string = input('\n Search for player: ')
        search = df[(df['shortName'].str.contains(player_string, case=False))][['playerId','shortName', 'team', 'position', 'marketValue']]
        if search.shape[0]==0:
            print('\n No results found. Please try again. Do not include the first name of the player, but only his last name or nickname.')
            j=j
        else:
            j=j+1
    display(search)
    playerId = input('\n Enter the playerId of the player you were looking for, or enter 0 for a new search: ')
    try:
        playerId = int(playerId)
    except:
        playerId = input('\n Please enter an integer number (playerId): ')
        playerId = int(playerId)
    if playerId==0:
        i=i
    else:
            i=i+1
value = input("\n Look for similar players for less than (in millions of euros, type 'Any' if no money filter is desired): ")
if (value!='Any') and (value!='any'):
    try:
        value = float(value)
        value = value*1000000
    except:
        value = input('\n Please enter a number (market value in millions of Euros): ')
        value = float(value)
        value = value*1000000
        
if df[df['playerId']==playerId]['playerId'].any():
    print('\nComputing Similarity. Please wait...')
    df=normalize(df)
    similarity = compute_similarity(df,playerId,900)
    print(color.BOLD + "\nThe 5 players most similar to {} for less than {} Euros are:".format(
                                                        df[df['playerId']==playerId]['shortName'].iloc[0], value) + color.END)
    if (value!='Any') and (value!='any'):
        similarity = similarity[similarity['value']<=value]
        aux = similarity.reset_index()
        num = aux.iloc[0,aux.columns.get_loc('index')]
        print('(Note: There are {} players more similar to {} than {} for a higher price)'.format(
                                    num, all_players[all_players['playerId']==playerId]['shortName'].iloc[0], aux.iloc[0,2]))
    show = similarity[['playerId', 'shortName', 'team', 'position', 'age','height', 'foot', 'goals', 'minutesPlayed', 'SimilarityIndex']].reset_index(drop=True)
    display(show.head(5))
else:
    print('\nPlayer does not exist or did not play the minimum 900 minutes across the 2017-18 season. \n\
    Please try a different player.')


 Search for player: messi


,playerId,shortName,team,position,marketValue
91,3359,L. Messi,Barcelona,Forward,"$180,000,000"



 Enter the playerId of the player you were looking for, or enter 0 for a new search: 3359

 Look for similar players for less than (in millions of euros, type 'Any' if no money filter is desired): any

Computing Similarity. Please wait...

The 5 players most similar to L. Messi for less than any Euros are:


,playerId,shortName,team,position,age,height,foot,goals,minutesPlayed,minutesPlayed,SimilarityIndex
0,89186,P. Dybala,Juventus,Forward,24,177,left,16,2359,2359,88.298
1,25707,E. Hazard,Chelsea,Forward,27,173,right,10,2432,2432,86.228
2,20820,J. Ilicic,Atalanta,Forward,30,190,left,9,2002,2002,85.554
3,3840,Iago Aspas,Celta de Vigo,Forward,31,176,left,19,2927,2927,85.279
4,28115,N. Fekir,Olympique Lyonnais,Midfielder,25,173,left,12,2482,2482,85.199
